In [1]:
#importa os pacotes necessarios
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#Cria metodo para listar as temporadas que existe no site
def lista_temporada(url_serie):
  url_base = url_serie
  req = requests.get(url_base)


  soup = BeautifulSoup(req.text, 'html.parser')
  lista_temporada_anos = []


  for a in soup.find_all('option'):
    lista_temporada_anos.append(url_base+a.get_text('Value'))
  
  return sorted(lista_temporada_anos)

In [3]:
#Cria metodo para listar os links de cada jogo
def lista_link_jogo(link_temporada):
  url_base = link_temporada
  req = requests.get(url_base)


  soup = BeautifulSoup(req.text, 'html.parser')
  lista_link_jogo = []


  link_jogo = soup.find_all(class_ ='btn btn-xs btn-success m-t-5')
  for link in link_jogo:
    lista_link_jogo.append(link['href'])


  return lista_link_jogo

In [4]:
#Cria metodo para buscar e salvas os dados requiridos
def busca_dado_partida(link_partida):
  req = requests.get(link_partida)


  soup = BeautifulSoup(req.text, 'html.parser')


  serie = soup.find(class_ = 'color-white block').text.split('-')[1].strip().split(' ')[1]
  temporada = int(soup.find(class_ = 'color-white block').text.split('-')[2].strip()) 
  estadio = soup.find_all(class_ = 'text-2 p-r-20')[0].text.split('-')[0].strip() 
  cidade = soup.find_all(class_ = 'text-2 p-r-20')[0].text.split('-')[1].strip()
  estado = soup.find_all(class_ = 'text-2 p-r-20')[0].text.split('-')[2].strip()
  dia_semana = soup.find_all(class_ = 'text-2 p-r-20')[1].text.split(',')[0].strip()
  dia_numero = int(soup.find_all(class_ = 'text-2 p-r-20')[1].text.split(',')[1].strip().split(' ')[0])
  mes = soup.find_all(class_ = 'text-2 p-r-20')[1].text.split(',')[1].strip().split('de')[1]
  ano = int(soup.find_all(class_ = 'text-2 p-r-20')[1].text.split(',')[1].strip().split('de')[2]) 
  hora = soup.find_all(class_ = 'text-2 p-r-20')[2].text.strip()
  mandante = soup.find_all(class_ = 'time-nome color-white')[0].text.strip()
  visitante = soup.find_all(class_ = 'time-nome color-white')[1].text.strip() 
  gol_mandante = int(soup.find_all(class_ = 'time-gols block')[0].text.strip())
  gol_visitante = int(soup.find_all(class_ = 'time-gols block')[1].text.strip())


  if gol_mandante > gol_visitante:
    vitoria_mandante = 1
    vitoria_visitante = 0
    jogo_empatado = 0
    ponto_mandante = 3
    ponto_visitante = 0
  if gol_mandante < gol_visitante:
    vitoria_mandante = 0
    vitoria_visitante = 1
    jogo_empatado = 0
    ponto_mandante = 0
    ponto_visitante = 3
  if gol_mandante == gol_visitante:
    vitoria_mandante = 0
    vitoria_visitante = 0
    jogo_empatado = 1
    ponto_mandante = 1
    ponto_visitante = 1


  # Cria um dicionario para estruturar os dados
  partida = {
      'serie' : serie,
      'temporada' : temporada,
      'estadio' : estadio,
      'cidade' : cidade,
      'estado' : estado,
      'dia_semana' : dia_semana,
      'dia_numero' : dia_numero,
      'mes' : mes,
      'ano' : ano,
      'hora' : hora,
      'mandante' : mandante,
      'visitante' : visitante,
      'gol_mandante' : gol_mandante,
      'gol_visitante' : gol_visitante,
      'vitoria_mandante' : vitoria_mandante,
      'vitoria_visitante' : vitoria_visitante,
      'jogo_empatado' : jogo_empatado,
      'ponto_mandante' : ponto_mandante,
      'ponto_visitante' : ponto_visitante
  }
  return(partida)

In [7]:
# Executa todos os metos criados para extração
url_serie = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a/'
dados = []


for link_temporada in lista_temporada(url_serie):
  print(link_temporada)
  for link_partida in lista_link_jogo(link_temporada):
    print(busca_dado_partida(link_partida))
    dados.append(busca_dado_partida(link_partida))

https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a/2012
{'serie': 'A', 'temporada': 2012, 'estadio': 'Pacaembu', 'cidade': 'Sao Paulo', 'estado': 'SP', 'dia_semana': 'Sábado', 'dia_numero': 19, 'mes': ' Maio ', 'ano': 2012, 'hora': '18:30', 'mandante': 'Palmeiras - SP', 'visitante': 'Portuguesa - SP', 'gol_mandante': 1, 'gol_visitante': 1, 'vitoria_mandante': 0, 'vitoria_visitante': 0, 'jogo_empatado': 1, 'ponto_mandante': 1, 'ponto_visitante': 1}
{'serie': 'A', 'temporada': 2012, 'estadio': 'Ilha do Retiro', 'cidade': 'Recife', 'estado': 'PE', 'dia_semana': 'Sábado', 'dia_numero': 19, 'mes': ' Maio ', 'ano': 2012, 'hora': '18:30', 'mandante': 'Sport - PE', 'visitante': 'Flamengo - RJ', 'gol_mandante': 1, 'gol_visitante': 1, 'vitoria_mandante': 0, 'vitoria_visitante': 0, 'jogo_empatado': 1, 'ponto_mandante': 1, 'ponto_visitante': 1}
{'serie': 'A', 'temporada': 2012, 'estadio': 'Orlando Scarpelli', 'cidade': 'Florianopolis', 'estado': 'SC', 'dia_semana':

KeyboardInterrupt: 

In [ ]:
#Salva os dados em um arquivo csv
df = pd.DataFrame.from_dict(dados)
# Salvar os dados combinados em CSV
df.to_csv('dados-brasileirao.csv', index=False)

Web Scraping feito com essa referência: https://pt.linkedin.com/pulse/extraindo-dados-do-campeonato-brasileiro-site-da-cbf-alexandre